In [ ]:
# %load trafficlib.py
import sys
import numpy as np

def initroad(road, density, seedval):

    np.random.seed(seedval)

    n = len(road)-2

    rng = np.random.random(n)

    road[1:n+1] = np.where(rng[:] < density, 1, 0)

    ncar = np.sum(road[1:n+1])
    
    return ncar


def updateroad(newroad, oldroad):

    n = len(oldroad)-2

    newroad[1:n+1] = np.where(oldroad[1:n+1]==0, oldroad[0:n], oldroad[2:n+2])

    nmove = (newroad[1:n+1] != oldroad[1:n+1]).sum(dtype=int)
    nmove = nmove / 2

    return nmove


def updatebcs(road):

    n = len(road)-2

    road[0]   = road[n]
    road[n+1] = road[1]


import time

def gettime():

    return time.time()


In [1]:
# %load traffic.py
#!/usr/bin/env python

import sys
import time
import numpy as np

from trafficlib import initroad, updatebcs, updateroad, gettime

def main(argv):

    # Simulation parameters
    seedval = 5743
    ncell = 10240000
    maxiter = 1024000000//ncell
    printfreq = maxiter//10

    newroad  = np.zeros(ncell+2, dtype=np.int32)
    oldroad  = np.zeros(ncell+2, dtype=np.int32)

    density = 0.52

    print(f"Length of road is {ncell}")
    print(f"Number of iterations is {maxiter}")
    print(f"Target density of cars is {density}")

    # Initialise road accordingly using random number generator
    print(f"Initialising ...")

    ncars = initroad(oldroad, density, seedval)

    print(f"Actual Density of cars is {format(float(ncars)/float(ncell))}\n")

    tstart = gettime()

    for iter in range(1, maxiter+1):

        updatebcs(oldroad)

        nmove = updateroad(newroad, oldroad)

        # Copy new to old array
        oldroad[1:ncell+1] = newroad[1:ncell+1]

        if iter % printfreq == 0:

          print(f"At iteration {iter} average velocity is {float(nmove)/float(ncars):.6f}")

    tstop = gettime()

    print(f"\nFinished\n")
    print(f"Time taken was {tstop-tstart} seconds")
    print(f"Update rate was {1.0e-6*ncell*maxiter/(tstop-tstart)} MCOPs\n")

if __name__ == "__main__":
    main(sys.argv[1:])


Length of road is 10240000
Number of iterations is 100
Target density of cars is 0.52
Initialising ...
Actual Density of cars is 0.520013671875

At iteration 10 average velocity is 0.789451
At iteration 20 average velocity is 0.837291
At iteration 30 average velocity is 0.858276
At iteration 40 average velocity is 0.870672
At iteration 50 average velocity is 0.879025
At iteration 60 average velocity is 0.885042
At iteration 70 average velocity is 0.889747
At iteration 80 average velocity is 0.893434
At iteration 90 average velocity is 0.896450
At iteration 100 average velocity is 0.898941

Finished

Time taken was 8.008042812347412 seconds
Update rate was 127.87144424616694 MCOPs

